In [1]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
from read_data import read_data_to_df

path = "C:\\Users\\grego\\interactivebrockers\\data\\EurGbpFx.txt"
df =  read_data_to_df(path)

fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

In [2]:
import random
random.seed(1983)
idle_time = [random.randrange(1,10) for x in range(0,10)]
open_time = [random.randrange(1,5) for p in range(0,10)]

start_time = 300

def calculate_start_end(start, idle_times, open_times):
    """
    Function calculates start and end times based on idle and open_times intervals
    """
    n_intervals = len(idle_times)
    current_time = start
    positions = []
    for i in range(n_intervals):
        buy_time = current_time + idle_times[i]
        sell_time = buy_time + open_times[i]
        current_time = sell_time
        positions.append([buy_time, sell_time])

    return positions

positions = calculate_start_end(start_time, idle_time, open_time)
print(idle_time)
print(open_time)
print(positions)


[2, 1, 3, 8, 6, 6, 9, 2, 6, 8]
[3, 1, 4, 2, 2, 3, 2, 2, 4, 3]
[[302, 305], [306, 307], [310, 314], [322, 324], [330, 332], [338, 341], [350, 352], [354, 356], [362, 366], [374, 377]]


In [3]:
def get_prices(pair, data):
    open_time = data.loc[pair[0],"date"]
    close_time = data.loc[pair[1],"date"]
    buy_price = data.loc[pair[0],"open"]
    sell_price = data.loc[pair[1],"close"]
    profit = buy_price - sell_price
    return [open_time, close_time, buy_price, sell_price, profit]

lines = pd.DataFrame([get_prices(x, df) for x in positions],
                     columns=["open_time","close_time","buy_price","sell_price","profit"])

print(lines)

            open_time          close_time  buy_price  sell_price    profit
0 2020-12-11 04:17:00 2020-12-11 04:20:00   0.912625    0.912525  0.000100
1 2020-12-11 04:21:00 2020-12-11 04:22:00   0.912525    0.912675 -0.000150
2 2020-12-11 04:25:00 2020-12-11 04:29:00   0.912610    0.912595  0.000015
3 2020-12-11 04:37:00 2020-12-11 04:39:00   0.912605    0.912650 -0.000045
4 2020-12-11 04:45:00 2020-12-11 04:47:00   0.912820    0.912755  0.000065
5 2020-12-11 04:53:00 2020-12-11 04:56:00   0.912800    0.912760  0.000040
6 2020-12-11 05:05:00 2020-12-11 05:07:00   0.912870    0.912755  0.000115
7 2020-12-11 05:09:00 2020-12-11 05:11:00   0.912720    0.912680  0.000040
8 2020-12-11 05:17:00 2020-12-11 05:21:00   0.912640    0.912835 -0.000195
9 2020-12-11 05:29:00 2020-12-11 05:32:00   0.912940    0.912980 -0.000040


In [4]:
round(lines.profit.sum(),4)*1000

-0.1

Now let's make them do this operation x times

In [12]:
def evaluate_permutation(data, n_trades, investment=1000):
    """
    function generates a random permutation and calculates its profit or loss
    :data: data frame containing stock information
    :n_trades: number of trades to be generated for each permutaion
    :return: a single number containing cumulative profit or loss
    """
    start = random.randint(1, data.shape[0]-200)
    idle_time = [random.randrange(1,10) for x in range(0,n_trades)]
    open_time = [random.randrange(1,5) for p in range(0,n_trades)]

    start_end_pair = calculate_start_end(start, idle_time, open_time)
    lines = pd.DataFrame([get_prices(x, data) for x in start_end_pair],
                         columns=["open_time","close_time","buy_price","sell_price","profit"])
    lines["profit_per_investment"] = lines.profit*investment
    return round(lines.profit_per_investment,4)

random.seed(1983)
profits = []
for i in range(100):
    profits.append(evaluate_permutation(df, 10))

profits = [round(x,6) for x in profits]
print(profits)

[0.455, 0.635, -0.715, -0.7, 1.815, 0.33, 1.03, 0.92, 0.29, 0.93, 0.515, 1.39, 0.155, 0.18, -0.205, -0.42, 0.575, -0.445, 1.29, -0.02, -0.21, -1.685, -1.015, 0.42, 1.72, -0.025, 0.115, -0.555, -0.82, 3.75, -0.1, 2.135, 0.73, -0.435, -0.22, 4.25, -0.435, -1.28, -1.005, -0.475, 0.25, -0.07, -0.44, -0.555, 0.21, -4.44, -2.905, -0.065, -0.105, -0.205, -0.435, 1.115, -0.08, -0.655, 0.23, -0.59, -0.555, 0.925, -0.145, 1.875, -0.19, 0.08, -0.55, 0.11, -1.13, -1.845, -0.195, -0.385, 0.005, 0.565, -1.155, 0.085, 0.16, 1.42, -0.095, 1.63, 0.25, -0.325, -0.3, 0.2, 0.05, 1.475, -1.48, -0.765, -1.6, -0.45, 1.16, -0.365, 0.37, 0.585, 1.02, -2.355, -0.32, -0.08, -0.505, 0.065, 0.57, -0.49, -0.19, 0.1]


In [13]:
import plotly.figure_factory as ff
import numpy as np
fig = ff.create_distplot([profits], ["Profit per iteration"], bin_size=.2)
fig.show()

So what are characteristics of this dummy algo?
givent that initial investissement is 1000 EUR

In [15]:
import statistics as s

print(f"Mean: {s.mean(profits)},\tSD: {s.stdev(profits)},\t sum: {sum(profits)}")

Mean: 0.013549999999999994,	SD: 1.1281983257039572,	 sum: 1.3550000000000002


If we limit our loss to 0.02

In [20]:
profits_after_stopp_loss = [-0.02 if x < -0.02 else x for x in profits ]
fig = ff.create_distplot([profits_after_stopp_loss], ["Profit per iteration"], bin_size=.1)
fig.show()

In [19]:
print(f"Mean: {s.mean(profits_after_stopp_loss)},\tSD: {s.stdev(profits_after_stopp_loss)},\t sum: {sum(profits_after_stopp_loss)}")

Mean: 0.37055,	SD: 0.7390677635311319,	 sum: 37.054999999999986
